In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud ,STOPWORDS
from collections import Counter

In [54]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [55]:
print('train shape:', train.shape)
train.head(10)

train shape: (159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\r\n\r\nCongratulations from me as well, use ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [56]:
print('test shape:', test.shape)
train.head()

test shape: (153164, 2)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [57]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [58]:
print(len(train))

159571


In [59]:
import random

N=list(range(len(train)))
random.seed(2023)
random.shuffle(N)
train=train.iloc[N[0:10000]]
display(train[0:3])
print(train.columns.tolist())
class_names=['No','Yes']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
56563,9715b0b10b11cdd3,I know it's more like I don't appereciate it ...,0,0,0,0,0,0
97499,09a4228af625232a,Recipients of the Lithuanian National Prize\r\...,0,0,0,0,0,0
1005,02bc73f81346f703,Perhaps you should better do self a better job...,0,0,0,0,0,0


['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [60]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [61]:
import string
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english') + ['u', 'im', 'c'])

def preprocess_text(sen):
    sentence = sen.lower()
    sentence = re.sub('\[.*?\]', '', sentence)
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    sentence = re.sub('\w*\d\w*', '', sentence)
    sentence = sentence.strip()
    # Удаление символов пунктуации и специальных символов
    sentence = re.sub('[^a-zA-Z0-9]', ' ', sentence)
    # Заменяем любые пробелы на один обычный
    sentence = re.sub(r'\s+', ' ', sentence)
    # Делаем всё в нижнем регистре
    sentence = sentence.lower()
    # Удаляем стоп-слова
    sentence = " ".join([word for word in sentence.split() if word not in stop_words])
    return sentence

train['clean_text'] = train['comment_text'].apply(preprocess_text)
test['clean_text'] = test['comment_text'].apply(preprocess_text)
#
# filtered_train = train['comment_text'].apply(preprocess_text)
# filtered_test = test['comment_text'].apply(preprocess_text)

print(stop_words)

train.head(10)

{'again', "doesn't", 'out', "needn't", 'before', 'were', 'does', 'she', 'other', 'such', 'off', 'just', 'that', 'why', 'some', 'u', 'which', 'by', 'shouldn', 'in', 's', 'with', 'll', 'me', 'herself', "hadn't", 'yours', 'theirs', "shan't", 'what', 'once', 'down', "you'll", 'its', 'hers', 'there', 'am', 'we', 'it', 'this', 'won', 'as', 'i', 'above', 'own', 'yourselves', 'wasn', "should've", 'don', 'below', 'where', 'aren', 'ours', "you'd", "shouldn't", "mightn't", 'd', 'didn', "she's", 'have', 'couldn', 'more', 'themselves', 'now', 'very', 'few', 'ourselves', 'can', "wouldn't", 'our', "it's", 'y', "aren't", 'under', 'is', 'him', 'to', 'each', 'here', 'mightn', 'any', 'o', 'than', 'hasn', 'and', 'you', 're', 'those', 'so', "hasn't", "you're", "weren't", 'yourself', 'having', 'im', 'no', 'same', 'the', 'during', 'nor', 'against', 'are', 'itself', 'needn', 'they', 'c', 'should', 'on', 'your', 'haven', 'only', 'be', 'been', 'both', "wasn't", 'ma', 'not', 'weren', 'has', 'my', 'do', 'too', 'w

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
84569,e2426864d64a35ed,Please do not add nonsense to Wikipedia. It is...,0,0,0,0,0,0,please add nonsense wikipedia considered vanda...
63586,aa25e67563f8fda8,Why is okay for subject to substantially contr...,0,0,0,0,0,0,okay subject substantially contribute bio seri...
62666,a7b09d6f99d3a768,This is a very big part of Porsche's history t...,0,0,0,0,0,0,big part porsches history cannot overlooked su...
120590,8518ed74c778a274,"It's not a good article, doesn't matter what t...",0,0,0,0,0,0,good article doesnt matter people want doesnt ...
108481,43f879832b485266,October 2007 (UTC)\r\nI didn't know they let o...,0,0,0,0,0,0,october utc didnt know let old ladies offshore...
119507,7ee9d30e23b8bbca,"""\r\n\r\nSpeedy deletion of Anna Hutchison\r\n...",0,0,0,0,0,0,speedy deletion anna hutchison tag placed anna...
35394,5e84a2a70300ef8b,Please do not add inappropriate external links...,0,0,0,0,0,0,please add inappropriate external links wikipe...
144015,0667f1ed1f0dca4e,I'm denying BLP violations.99.141.241.135,0,0,0,0,0,0,denying blp
121019,8777f4b2809f4938,"""\r\n\r\n Surnames of allegèdly Bukhar[i]an Je...",0,0,0,0,0,0,surnames alleg dly bukharan jews one mystified...
50602,874c6d2db722ed23,"""\r\n====\r\nsee: A mystical view of the resur...",0,0,0,0,0,0,see mystical view resurrection dead within jud...


In [62]:
def remove_stopwords(text):
    words = text.split(' ')
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

train['clean_text'] = train['clean_text'].apply(remove_stopwords)
test['clean_text'] = test['clean_text'].apply(remove_stopwords)

In [63]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
84569,e2426864d64a35ed,Please do not add nonsense to Wikipedia. It is...,0,0,0,0,0,0,please add nonsense wikipedia considered vanda...
63586,aa25e67563f8fda8,Why is okay for subject to substantially contr...,0,0,0,0,0,0,okay subject substantially contribute bio seri...
62666,a7b09d6f99d3a768,This is a very big part of Porsche's history t...,0,0,0,0,0,0,big part porsches history cannot overlooked su...
120590,8518ed74c778a274,"It's not a good article, doesn't matter what t...",0,0,0,0,0,0,good article doesnt matter people want doesnt ...
108481,43f879832b485266,October 2007 (UTC)\r\nI didn't know they let o...,0,0,0,0,0,0,october utc didnt know let old ladies offshore...
119507,7ee9d30e23b8bbca,"""\r\n\r\nSpeedy deletion of Anna Hutchison\r\n...",0,0,0,0,0,0,speedy deletion anna hutchison tag placed anna...
35394,5e84a2a70300ef8b,Please do not add inappropriate external links...,0,0,0,0,0,0,please add inappropriate external links wikipe...
144015,0667f1ed1f0dca4e,I'm denying BLP violations.99.141.241.135,0,0,0,0,0,0,denying blp
121019,8777f4b2809f4938,"""\r\n\r\n Surnames of allegèdly Bukhar[i]an Je...",0,0,0,0,0,0,surnames alleg dly bukharan jews one mystified...
50602,874c6d2db722ed23,"""\r\n====\r\nsee: A mystical view of the resur...",0,0,0,0,0,0,see mystical view resurrection dead within jud...


In [71]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()

trainXV = vectorizer.fit_transform(train['clean_text'])
trainY = train['toxic']

testXV = vectorizer.transform(test['clean_text'])
testY = test['toxic']
print(trainXV.shape,testXV.shape)

(8000, 31617) (2000, 31617)


In [73]:
trainX = pd.DataFrame(trainXV.toarray(), columns=vectorizer.get_feature_names_out())
testX = pd.DataFrame(testXV.toarray(), columns=vectorizer.get_feature_names_out())
print(type(trainXV),type(trainX))

<class 'scipy.sparse._csr.csr_matrix'> <class 'pandas.core.frame.DataFrame'>


In [74]:
train=(trainX,trainY)
test=(testX,testY)